# <center><span style="color:#6699FF;"> $\mathit{Introduction}$</span><center>
* In this notebook, I will find all the images that has a valid rle code. For your imformation, I didn't use much advanced API or some complicated function, for it's easier to learners to comprehend.    

<h2> <span style="color:#CC66FF;"> Imports : </span></h2> 

In [ ]:
!pip install monai -q
!pip install imio -q

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import random
import skimage
from glob import glob
from skimage import io
from skimage import color
from imio import load, save
from monai.transforms import (
    Compose,
    AddChanneld,
    LoadImaged,
    Spacingd,
    Resized,
    ScaleIntensityRanged,
    CropForegroundd,
    Resized,
    ToTensor,
    ToTensord,
    Orientationd,
    NormalizeIntensityd,
    RandRotated,
    RandAxisFlipd,
    RandFlipd,
    AdjustContrastd,
)

<h2> <span style="color:#CC66FF;"> Search the images : </span></h2> 

* As we can see, there are many **NaNs** in the segmentation column. Therefore, we need to find out the rows which has an valid rle_code.

In [ ]:
train = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')
train.head(10)

In [ ]:
# Find the target rows
# We will use the index of the target rows
nan_list = list(train['segmentation'].isnull())
not_nan_index = []

for i in range(len(nan_list)):
    if nan_list[i] == False:
        not_nan_index.append(i)

# Let's see how many indice we got
print('The number of index is:', len(not_nan_index))

In [ ]:
# There are only 3 classes
all_class = ['stomach', 'small_bowel', 'large_bowel']

# The empty list below will be used to store all the indice of each class
stomach = []
small_bowel = []
large_bowel = []

for i in range(len(not_nan_index)):
    row = not_nan_index[i]
    if train.loc[row, 'class'] == all_class[0]:
        stomach.append(not_nan_index[i])
    elif train.loc[row, 'class'] == all_class[1]:
         small_bowel.append(not_nan_index[i])
    else:
        large_bowel.append(not_nan_index[i])

In [ ]:
# see how many images there are in each class
print(len(stomach))
print(len(small_bowel))
print(len(large_bowel))

In [ ]:
# get all ids of each class
stomach_ids = []
small_bowel_ids = []
large_bowel_ids = []

for i in range(len(stomach)):
    stomach_ids.append(train.loc[stomach[i], 'id'])
    
for i in range(len(small_bowel)):
    small_bowel_ids.append(train.loc[small_bowel[i], 'id'])
    
for i in range(len(large_bowel)):
    large_bowel_ids.append(train.loc[large_bowel[i], 'id'])
    
# demonstrate some ids
print(stomach_ids[0:5])

# Experiment

* The **Experiment** means I'm doing an experiment in the below cells, It's an additional part.

* Here's the general steps:
1. First, I found all images that has a valid rle_code in the entire original dataset, and then I split thoes images into 3 classes, I did that in the cell above.
2. In this step, we will find all images that has 3 different rle_code. So I use the ids in the list **stomach** to search the same ids in the other 2 lists (**small_bowel, large_bowel**), then I stored all these images in the **correspond** list. Now each image in correspond has 3 different masks.

In [ ]:
correspond = []

for i in range(len(stomach_ids)):
    image_id = stomach_ids[i]
    if (image_id in small_bowel_ids) & (image_id in large_bowel_ids):
        correspond.append(image_id)

As you can see, I only found about 3000 images.

In [ ]:
len(correspond)

In [ ]:
# I put a "1" at the end to distinguish them with the variable name in the original notebook.
# In this step, I will find the corresponding index of each image, so I can use the index to locate the target image in train.csv

stomach_1 = []
small_bowel_1 = []
large_bowel_1 = []

for i in range(len(list(not_nan_index))):
    if (train['id'].iloc[not_nan_index[i]] in correspond) & (train['class'].iloc[not_nan_index[i]] == 'stomach'):
        stomach_1.append(not_nan_index[i])
        
for i in range(len(list(not_nan_index))):
    if (train['id'].iloc[not_nan_index[i]] in correspond) & (train['class'].iloc[not_nan_index[i]] == 'small_bowel'):
        small_bowel_1.append(not_nan_index[i])
        
for i in range(len(list(not_nan_index))):
    if (train['id'].iloc[not_nan_index[i]] in correspond) & (train['class'].iloc[not_nan_index[i]] == 'large_bowel'):
        large_bowel_1.append(not_nan_index[i])

In [ ]:
# These are helper functions 

src = '../input/uw-madison-gi-tract-image-segmentation/train'

# This dictionary stores the folders'name in each case. (e.g. case101_day20)
case_day = {}

all_case = os.listdir(src)
for i in range(len(all_case)):
    path = src + '/' + all_case[i]
    day = os.listdir(path)
    case_day[all_case[i]] = day


def filename_generator(ID):
    split_id = ID.split('_')
    case = split_id[0]
    day_list = case_day[case]
    day = []
    for d in range(len(day_list)):
        day.append(day_list[d].split('_')[1])
    for k in range(len(day)):
        if split_id[1] == day[k]:
            dr = src + '/' + case + '/' + case + '_' + split_id[1] + '/' + 'scans'
            h = os.listdir(dr)[0].split('_')[2]
            w = os.listdir(dr)[0].split('_')[3]
            p = os.listdir(dr)[0].split('_')[4]
            break
    filename = src + '/' + split_id[0] + '/' + split_id[0] + '_' + split_id[1] + '/' + 'scans' + '/' + split_id[2] + '_' + split_id[3] + '_' + h + '_' + w + '_' + p + '_' + p + '.png'
    size = [h, w]
    return filename, size

def load_image(filename):
    img = cv2.imread(filename, cv2.CV_16UC1)
    return img

def display(display_list):
    plt.figure(figsize=(10, 10))
    
    title = ['image', 'image with mask']

    for i in range(len(display_list)):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()
    
def rle_decode(rle, height, width , fill=255):
    s = rle.split()
    start, length = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    start -= 1
    mask = np.zeros(height*width, dtype=np.uint8)
    for i, l in zip(start, length):
        mask[i:i+l] = fill
    mask = mask.reshape(width,height).T
    mask = np.ascontiguousarray(mask)
    return mask

In [ ]:
fname_1 = []
size_1 = []
stomach_mask_1 = []
small_bowel_mask_1 = []
large_bowel_mask_1 = []

for i in range(len(correspond)):
    ID = correspond[i]
    filename = filename_generator(ID)
    fname_1.append(filename[0])
    size_1.append(filename[1])
    

for i in range(len(stomach_1)):
    index = stomach_1[i]
    rle = train['segmentation'].iloc[index]
    stomach_mask_1.append(rle)
    
for i in range(len(small_bowel_1)):
    index = small_bowel_1[i]
    rle = train['segmentation'].iloc[index]
    small_bowel_mask_1.append(rle)
    
for i in range(len(large_bowel_1)):
    index = large_bowel_1[i]
    rle = train['segmentation'].iloc[index]
    large_bowel_mask_1.append(rle)

In [ ]:
print(len(fname_1))
print(len(size_1))
print(len(stomach_mask_1))
print(len(small_bowel_mask_1))
print(len(large_bowel_mask_1))

In [ ]:
info_dict_1 = {'id':correspond, 
             'path':fname_1, 
             'stomach_mask':stomach_mask_1,
             'small_bowel_mask':small_bowel_mask_1,
             'large_bowel_mask':large_bowel_mask_1,
              'size':size_1}
data_1 = pd.DataFrame(info_dict_1,  columns=['id', 'path', 'stomach_mask', 'small_bowel_mask', 'large_bowel_mask', 'size'])
data_1.head()

* I do the label job in this cell, I will show you how I did: When using the **rle_decode** function, we will get a array which has only 2 values (0, 1), this works for all masks. While decoding different rle_code, the only thing that will change is the position of 0 and 1. Therefore, I used numpy API to switch the 1 with other numbers (i.e. 1 for stomach, 2 for small_bowel, 3 for large_bowel). Finally, I added all the three arrays together to acquire a mask array with all 3 masks.

In [ ]:
fnames = list(data_1['path'])
stomach_masks = list(data_1['stomach_mask'])
small_bowel_masks = list(data_1['small_bowel_mask'])
large_bowel_masks = list(data_1['large_bowel_mask'])
size = list(data_1['size'])
display_list = []

M = []

for i in range(10):
    img = load_image(fnames[i])
    h = int(size[i][0])
    w = int(size[i][1])
    stomach_mask = rle_decode(rle = stomach_masks[i], height = h, width = w)
    small_bowel_mask = rle_decode(rle = small_bowel_masks[i], height = h, width = w)
    large_bowel_mask = rle_decode(rle = large_bowel_masks[i], height = h, width = w)
    stomach_mask = (stomach_mask / 255).astype(int).T
    small_bowel_mask = (small_bowel_mask / 255).astype(int).T
    small_bowel_mask[np.where(small_bowel_mask==1)]=2
    large_bowel_mask = (large_bowel_mask / 255).astype(int).T
    large_bowel_mask[np.where(large_bowel_mask==1)]=3
    all_mask = stomach_mask + small_bowel_mask + large_bowel_mask
    result_image_0 = color.label2rgb(stomach_mask, img / img.max())
    result_image_1 = color.label2rgb(small_bowel_mask, img / img.max())
    result_image_2 = color.label2rgb(large_bowel_mask, img / img.max())
    result_image_3 = color.label2rgb(all_mask, img / img.max())
    display_list.append([img, result_image_0, result_image_1, result_image_2, result_image_3])
    M.append(all_mask)

In [ ]:
np.unique(M[0])

In [ ]:
def display_1(display_list):
    plt.figure(figsize=(10, 10))
    
    title = ['image', 'stomach mask', 'small_bowel_mask', 'large_bowel_mask', 'all_mask']

    for i in range(len(display_list)):
        plt.subplot(1, 5, i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()
    
for i in range(10):
    display_1(display_list[i])

In [ ]:
!pip install imio -q
from imio import load, save
import tensorflow as tf

In [ ]:
image_path = list(data_1['path'])

# Creating Dir to store the converted files
dir = './dataset'
mask_out_path = './dataset/mask'
input_out_path = './dataset/image'
os.mkdir(dir)
os.mkdir(mask_out_path)
os.mkdir(input_out_path)

In [ ]:
img_path = './dataset/image'
mask_path = './dataset/mask'
ids = list(data_1['id'])

masks = []

for i in range(len(image_path)):
    img = load_image(image_path[i])
    h = int(size[i][0])
    w = int(size[i][1])
    stomach_mask = rle_decode(rle = stomach_masks[i], height = h, width = w)
    small_bowel_mask = rle_decode(rle = small_bowel_masks[i], height = h, width = w)
    large_bowel_mask = rle_decode(rle = large_bowel_masks[i], height = h, width = w)
    stomach_mask = (stomach_mask / 255).astype(int).T
    small_bowel_mask = (small_bowel_mask / 255).astype(int).T
    small_bowel_mask[np.where(small_bowel_mask==1)]=2
    large_bowel_mask = (large_bowel_mask / 255).astype(int).T
    large_bowel_mask[np.where(large_bowel_mask==1)]=3
    all_mask = stomach_mask + small_bowel_mask + large_bowel_mask
    masks.append(all_mask)

# old

In [ ]:
src = '../input/uw-madison-gi-tract-image-segmentation/train'

# This dictionary stores the folders'name in each case. (e.g. case101_day20)
case_day = {}

all_case = os.listdir(src)
for i in range(len(all_case)):
    path = src + '/' + all_case[i]
    day = os.listdir(path)
    case_day[all_case[i]] = day
    
# This function is used to generate the filename of each image, and it also outputs the size of each image
def filename_generator(ID):
    split_id = ID.split('_')
    case = split_id[0]
    day_list = case_day[case]
    day = []
    for d in range(len(day_list)):
        day.append(day_list[d].split('_')[1])
    for k in range(len(day)):
        if split_id[1] == day[k]:
            dr = src + '/' + case + '/' + case + '_' + split_id[1] + '/' + 'scans'
            h = os.listdir(dr)[0].split('_')[2]
            w = os.listdir(dr)[0].split('_')[3]
            p = os.listdir(dr)[0].split('_')[4]
            break
    filename = src + '/' + split_id[0] + '/' + split_id[0] + '_' + split_id[1] + '/' + 'scans' + '/' + split_id[2] + '_' + split_id[3] + '_' + h + '_' + w + '_' + p + '_' + p + '.png'
    size = [h, w]
    return filename, size

# These empty list are used to store something useful, according to the name of each list 
stomach_fname = []
stomach_size = []
small_bowel_fname = []
small_bowel_size = []
large_bowel_fname = []
large_bowel_size = []
stomach_mask = []
small_bowel_mask = []
large_bowel_mask = []

# You can define some helper functions as a substitution for these for loops, but I think that didn't make much difference
for i in range(len(stomach_ids)):
    ID = stomach_ids[i]
    filename = filename_generator(ID)
    stomach_fname.append(filename[0])
    stomach_size.append(filename[1])
    
for i in range(len(small_bowel_ids)):
    ID = small_bowel_ids[i]
    filename = filename_generator(ID)
    small_bowel_fname.append(filename[0])
    small_bowel_size.append(filename[1])
  
for i in range(len(large_bowel_ids)):
    ID = large_bowel_ids[i]
    filename = filename_generator(ID)
    large_bowel_fname.append(filename[0])
    large_bowel_size.append(filename[1])
    
for i in range(len(stomach)):
    rle = train.loc[stomach[i], 'segmentation']
    stomach_mask.append(rle)
    
for i in range(len(small_bowel)):
    rle = train.loc[small_bowel[i], 'segmentation']
    small_bowel_mask.append(rle)
    
for i in range(len(large_bowel)):
    rle = train.loc[large_bowel[i], 'segmentation']
    large_bowel_mask.append(rle)

<h2> <span style="color:#CC66FF;"> Generate Dataframe : </span></h2> 

In [ ]:
# We will use a Dataframe to store all the imformation we need.(e.g. height, width, maks, etc.)

# These lists store the information we need
stomach_info = []
small_bowel_info = []
large_bowel_info = []

for i in range(len(stomach)):
    ID = stomach_ids[i]
    name = 'stomach'
    path = stomach_fname[i]
    mask = stomach_mask[i]
    size = stomach_size[i]
    stomach_info.append([name, ID, path, mask, size])
    
for i in range(len(small_bowel)):
    ID = small_bowel_ids[i]
    name = 'small_bowel'
    path = small_bowel_fname[i]
    mask = small_bowel_mask[i]
    size = small_bowel_size[i]
    small_bowel_info.append([name, ID, path, mask, size])
    
for i in range(len(large_bowel)):
    ID = large_bowel_ids[i]
    name = 'large_bowel'
    path = large_bowel_fname[i]
    mask = large_bowel_mask[i]
    size = large_bowel_size[i]
    large_bowel_info.append([name, ID, path, mask, size])

In [ ]:
print(len(stomach_info))
print(len(small_bowel_info))
print(len(large_bowel_info))

In [ ]:
# We need to concatenate these infos, then shuffle it.
info = stomach_info + small_bowel_info + large_bowel_info
random.shuffle(info)
print(len(info))